In [16]:
import pandas as pd
import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
from datasets import DatasetDict, load_from_disk, Dataset, concatenate_datasets
import re

In [17]:
import json
with open('/opt/ml/input/data/ai_hub/ko_nia_clue0529_squad_all.json','r') as f:
    clue0529 = json.load(f)
with open('/opt/ml/input/data/ai_hub/ko_nia_noanswer_squad_all.json','r') as f:
    noanswer = json.load(f)
with open('/opt/ml/input/data/ai_hub/ko_nia_normal_squad_all.json','r') as f:
    normal = json.load(f)

In [18]:
clue0529.keys()

dict_keys(['data', 'creator', 'version'])

In [19]:
len(clue0529['data'])

34500

In [20]:
clue0529['data'][0]

{'source': 4,
 'paragraphs': [{'qas': [{'classtype': 'work_why',
     'id': 'm6_409391-1',
     'answers': [{'text': '수도 동파', 'answer_start': 144}],
     'question': '재개발단지에 남아있는 주민들이 불편을 겪는 이유는 뭐야?',
     'clue': [{'clue_start': 173,
       'clue_text': '미처 이사를 가지 못한 거주민들은 새는 물을 대야로 바치거나 습기가 가득 찬 마루 바닥을 온풍기로 말리며 힘겨운 겨울을 나고 있다.'}]},
    {'classtype': 'work_why',
     'id': 'm6_409391-2',
     'answers': [{'text': '수도 동파', 'answer_start': 144}],
     'question': '재개발단지에 남아있는 주민들이 불편을 겪는 이유는 무엇이야?',
     'clue': [{'clue_start': 173,
       'clue_text': '미처 이사를 가지 못한 거주민들은 새는 물을 대야로 바치거나 습기가 가득 찬 마루 바닥을 온풍기로 말리며 힘겨운 겨울을 나고 있다.'}]},
    {'classtype': 'work_what',
     'id': 'm6_409392-1',
     'answers': [{'text': '현관등이 있는 천정에서 물이 새면서 전기 누전으로 연기가 나고, 고드름을 제거하다 실수로 베란다 창문마저 깨졌다. ',
       'answer_start': 290}],
     'question': '정모씨(79)는 수도 동파로 인해 어떤 피해를 입었어?',
     'clue': [{'clue_start': 354,
       'clue_text': '정씨는 &quot;이 엄동설한에 어디로 나가야 하는 지 막막하다&quot;며 &quot;여기 저기 물이 새 집 안에서도 신발을 신고 다녀

In [21]:
noanswer.keys()

dict_keys(['creator', 'version', 'data'])

In [22]:
len(noanswer['data'])

20030

In [23]:
noanswer['data'][0]

{'source': 2,
 'title': '2654.txt',
 'paragraphs': [{'qas': [{'classtype': 'work_who',
     'id': 'm3_253409-1',
     'question': '제너시스비비큐의 대표이사는 누구야?'},
    {'classtype': 'work_when',
     'id': 'm3_253410-1',
     'question': '제너시스 비비큐의 창립일은 언제야?'},
    {'classtype': 'work_where',
     'id': 'm3_253411-1',
     'question': '공정거래위원회는 어디에 사무실이 위치해있어?'},
    {'classtype': 'work_how',
     'id': 'm3_253412-1',
     'question': '공정거래위원회가 제너시스비비큐에게 내린 시정명령은 어떤 내용이야?'},
    {'classtype': 'work_how',
     'id': 'm3_253413-1',
     'question': '다른 회사는 상품권 발행비용 명목으로 수수료를 얼마씩 받고있어?'},
    {'classtype': 'work_why',
     'id': 'm3_253414-1',
     'question': '왜 제너시스비비큐는 상품권 수령을 강요하면서까지 상품권 수수료를 챙기려고 했어?'}],
   'context': '공정거래위원회는 상품권 발행비용 명목으로 고율의 발행수수료를 가맹점주에게 부담토록 한 제너시스비비큐(BBQ)에게 시정명령을 내렸다. 공정위에 따르면 제너시스비비큐는 2011년 9월부터 지난해 7월까지 가맹점이 소비자로부터 받은 상품권을 정산하면서 액면가의 10%를 발행수수료로 공제했다. 제너시스비비큐는 또 높은 수수료 부담을 피하기 위해 고객들로부터 상품권을 받지 않은 가맹점에는 가맹계약을 해지할 수 있다는 증명을 발송해 상품권 수령을 강요하기도 했다. 이를 통해 본사가 챙긴 상품권 수수료는 2

In [24]:
normal.keys()

dict_keys(['creator', 'version', 'data'])

In [25]:
len(normal['data'])

47314

In [145]:
# http://zeany.net/46
def preprocess_context(example):
    body = re.sub('<title.*?>.*?</title>', '', example, 0, re.I|re.S)
    body = re.sub('<table.*?>.*?</table>', '', body, 0, re.I|re.S)
    body = re.sub('<td.*?>.*?</td>', '', body, 0, re.I|re.S)
    body = re.sub('<li.*?>.*?</li>', '', body, 0, re.I|re.S)
    body = re.sub('<ol.*?>.*?</ol>', '', body, 0, re.I|re.S)
    text = re.sub('<.+?>', '', body, 0, re.I|re.S)
    text = re.sub('\[[0-9]+\]', ' ', text, 0, re.I|re.S)
    text = text.replace("[원본 편집]","")
    text = text.replace("[편집]","")
    text = text.replace("\\'","")
    text = text.replace("펼치기","")
    text = text.replace("버튼 클릭","")
    text = text.split("같이 보기")[0]
    text = text.split("참고 자료")[0]
    text = text.split("외부 링크")[0]
    text = text.split("원본 주소")[0]
    text = text.split("각주")[0]
    text = text.split("참고 문헌")[0]
    text = text.split("함께 보기")[0]
    text = text.split("관련 서적")[0]
    text = text.split("관련 항목")[0]
    space = re.sub('&nbsp;|&lt;|&gt;|&quot;|\t|\r|\n|\xa0|\u200b', ' ', text)
    one_space = re.sub(' +', ' ', space)
    if '@' in one_space:
        one_space = ".".join(one_space.split('.')[:-(['@'in c for c in one_space.split('.')[::-1]].index(True)+1)])+"."
    if one_space[::-1].find('.') == -1:
        return one_space
    if (one_space[::-1].find('.') != 0)|(one_space[::-1].find('.') != ')'):
        last_comma = one_space[:-(one_space[::-1].find('.'))]
        return last_comma
    return one_space

In [146]:
# http://zeany.net/46
def preprocess_answer(example):
    body = re.sub('<title.*?>.*?</title>', '', example, 0, re.I|re.S)
    body = re.sub('<table.*?>.*?</table>', '', body, 0, re.I|re.S)
    body = re.sub('<td.*?>.*?</td>', '', body, 0, re.I|re.S)
    body = re.sub('<li.*?>.*?</li>', '', body, 0, re.I|re.S)
    body = re.sub('<ol.*?>.*?</ol>', '', body, 0, re.I|re.S)
    text = re.sub('<.+?>', '', body, 0, re.I|re.S)
    text = re.sub('\[[0-9]+\]', ' ', text, 0, re.I|re.S)
    text = text.replace("[원본 편집]","")
    text = text.replace("[편집]","")
    text = text.replace("\\'","")
    text = text.replace("펼치기","")
    text = text.replace("버튼 클릭","")
    text = text.split("같이 보기")[0]
    text = text.split("참고 자료")[0]
    text = text.split("외부 링크")[0]
    text = text.split("원본 주소")[0]
    text = text.split("각주")[0]
    text = text.split("참고 문헌")[0]
    text = text.split("함께 보기")[0]
    text = text.split("관련 서적")[0]
    text = text.split("관련 항목")[0]
    space = re.sub('&nbsp;|&lt;|&gt;|&quot;|\t|\r|\n|\xa0|\u200b', ' ', text)
    one_space = re.sub(' +', ' ', space)
    return one_space

In [147]:
import json
whole_list = []
train_dict = {}
answer_dict = {}
c = 0
with open('/opt/ml/input/data/ai_hub/ko_nia_normal_squad_all.json','r') as f:
    train_data = json.load(f)
    for i in range(len(train_data['data'])):
        for j in range(len(train_data['data'][i]['paragraphs'][0]['qas'])):
            answers = train_data['data'][i]['paragraphs'][0]['qas'][j]
            answer_dict['answer_start'] = [len(preprocess_answer(train_data['data'][i]['paragraphs'][0]['context'][:answers['answers'][0]['answer_start']]).lstrip())]
            answer_dict['text'] = [preprocess_answer(answers['answers'][0]['text']).strip()]
            count_id = ('000000' + str(c))[-6:]
            train_dict['title']=train_data['data'][i]['title']
            train_dict['context']= preprocess_context(train_data['data'][i]['paragraphs'][0]['context']).strip()
            train_dict['question']=answers['question']
            train_dict['id']='aihub-{}'.format(count_id)
            train_dict['answers']=answer_dict
            train_dict['document_id']=int(answers['id'].split('_')[1].replace('-',''))
            train_dict['__index_level_0__']=0
            if len(answer_dict['text'][0])>0:
                if (len(train_dict['context']) <= 2000)&(len(answer_dict['text']) <= 80):
                    if len(train_dict['context']) > answer_dict['answer_start'][0]:
                        if train_dict['context'][answer_dict['answer_start'][0]] != answer_dict['text'][0][0]:
                            continue
                        else:
                            whole_list.append(train_dict)
                    else:
                        continue
                else:
                    continue
            else:
                continue
            answer_dict={}
            train_dict={}
            c = c + 1

In [148]:
len(whole_list)

100210

In [149]:
import random

In [150]:
whole_list[0]

{'title': '20180328000443000',
 'context': '‘화 있을진저.’ 예수님께서 진노합니다. 단순한 화가 아니라 그 자리에서 돌이키라는 애틋한 호소입니다. 하나님의 분노에는 항상 사랑이 담겨 있습니다. 예수님께서 진노하는 대상은 소경된 인도자입니다(16절). 소경이란 눈먼 자가 아니라 영적 세계를 보지 못하는 자를 말합니다. 당시 대표적 영적 소경은 ‘사두개인’이었습니다. 사두개인은 유대 종교지도자였으나 부활을 믿지 않았습니다. 부활을 믿지 않는 이들은 모든 것을 세상의 기준으로 봅니다. 성전보다 성전의 금을 더 크게 보고(16절), 제단보다 제단 위 예물을 더 귀중하게 봅니다(18절). 성전 안에는 금이 많았습니다. 언약궤, 등대 등 성전의 여러 기구들은 모두 금으로 만들어졌습니다. 그러나 그 금이 거룩한 건 성전 안에 있기 때문입니다. 하지만 영적 소경은 성전 안의 금도 세상의 금과 같이 대했습니다. 제단 위의 예물도 그렇습니다. 제단 위에서 하나님께 드려질 때 소나 양은 거룩한 제물이 됩니다. 그러나 영적 소경은 제단보다 제물을 더 귀중하게 봤습니다. 왜냐하면 소나 양이 제단보다 더 재산 가치가 높았기 때문입니다. 금보다 성전이 더 크다는 것을 보지 못하고 예물보다 제단이 더 귀한 것을 못 보는 자들, 이들이 영적 소경입니다. 이런 사람들이 인도자가 돼 백성을 인도하는 것에 예수님께서는 진노했습니다. 오늘날 상황도 이와 같습니다. 영적 소경이 인도자가 돼 이끄는 성도는 성전이 아니라 금을 좇게 됩니다. 금전만능주의, 성공제일주의, 축복신앙, 번영신학 같은 거짓 가르침을 따르게 됩니다. 따라서 성도는 눈을 뜨고 영적 인도자가 소경인지 아닌지를 점검해야 합니다. 예수님의 진노는 교회 지도자뿐 아니라 성도 모두에게 주신 경고입니다(마 23:1). 그렇다면 성도들이 봐야 할 성전과 제단은 무엇을 의미할까요. 성전은 바로 예수 그리스도를 가리킵니다(히 9:11). 따라서 눈뜬 사람은 예수님을 통해 얻는 금보다 예수님을 바라봅니다. 예수 

In [95]:
rand_idx=random.sample(range(0,len(whole_list)),len(whole_list)//10)

In [96]:
len(rand_idx)

14732

In [97]:
train_list=[]
dev_list=[]

In [98]:
for idx, w in enumerate(whole_list):
    if idx not in rand_idx:
        train_list.append(w)
    else:
        dev_list.append(w)

In [107]:
train_10=[]
dev_10=[]

In [108]:
for idx, w in enumerate(whole_list):
    if idx < 14732:
        if idx not in rand_idx:
            train_10.append(w)
        else:
            dev_10.append(w)

In [99]:
len(train_list)

132588

In [100]:
len(dev_list)

14732

In [ ]:
train_90=[]
dev_90=[]
for idx, w in enumerate(whole_list):
    if idx >= 14732:
        if idx not in rand_idx:
            train_90.append(w)
        else:
            dev_90.append(w)

In [101]:
whole_list[0]

{'title': '775',
 'context': "[헤럴드POP=고승아 기자]그룹 구구단 샐리가 리듬체조에서 실수했다.15일 방송된 MBC '설특집 2018 아이돌스타 육상 볼링 양궁 리듬체조 에어로빅 선수권대회(이하 아육대)'에서는 구구단 샐리가 리듬체조 종목에 출전한 모습이 그려졌다.이날 구구단 샐리는 리듬체조 종목에 처음 출전해 훌라후프로 무대를 꾸몄다. 그러나 연기 도중 훌라후프를 받지 못하는 등 실수를 해 안타까움을 자아냈다.응원을 하던 구구단 멤버들도 열심히 연습했던 샐리를 생각해 눈물을 흘렸다.popnews@heraldcorp.com- Copyrights ⓒ 헤럴드POP & heraldpop.",
 'question': '아육대에서 리듬체조에 출전한 구구단의 멤버는?',
 'id': 'aihub-000000',
 'answers': {'answer_start': [22], 'text': ['샐리']},
 'document_id': 3067051,
 '__index_level_0__': 0}

In [102]:
train_list[0]

{'title': '775',
 'context': "[헤럴드POP=고승아 기자]그룹 구구단 샐리가 리듬체조에서 실수했다.15일 방송된 MBC '설특집 2018 아이돌스타 육상 볼링 양궁 리듬체조 에어로빅 선수권대회(이하 아육대)'에서는 구구단 샐리가 리듬체조 종목에 출전한 모습이 그려졌다.이날 구구단 샐리는 리듬체조 종목에 처음 출전해 훌라후프로 무대를 꾸몄다. 그러나 연기 도중 훌라후프를 받지 못하는 등 실수를 해 안타까움을 자아냈다.응원을 하던 구구단 멤버들도 열심히 연습했던 샐리를 생각해 눈물을 흘렸다.popnews@heraldcorp.com- Copyrights ⓒ 헤럴드POP & heraldpop.",
 'question': '아육대에서 리듬체조에 출전한 구구단의 멤버는?',
 'id': 'aihub-000000',
 'answers': {'answer_start': [22], 'text': ['샐리']},
 'document_id': 3067051,
 '__index_level_0__': 0}

In [103]:
dev_list[0]

{'title': '20180312182928000',
 'context': '"미국 정부의 철강 관세 부과 움직임과 관련해 우리 정부가 총력 대응하겠다는 뜻을 밝혔습니다. 수출 시장을 넓히기 위해 일본 등 11개 국가가 참여하는 자유무역 협정인 &환태평양경제동반자협정& 가입 여부도 상반기 안에 결정할 방침입니다. 김경진 기자의 보도입니다. [리포트] 김동연 경제부총리는 오늘 열린 대외경제장관회의에서, 미국의 철강 관세 부과 움직임과 관련해 정부가 할 수 있는 모든 것을 다 하겠다는 입장을 다시 강조했습니다. [김동연/경제부총리 : 미국의 관세 부과 조치에 대해서는 정부의 모든 가용 채널을 활용해서 총력 대응해 나갈 계획입니다.] 정부는 한국산 철강의 관세 면제 필요성을 적극 설득하기 위해 므누신 미 재무장관에게 서한을 보냈고, 다음 주 G20 재무장관 회의에서도 이 문제에 대해 협의할 예정입니다. 정부는 또 미국이 보호 무역 정책에 적극적으로 대응하는 동시에 수출 시장은 더 넓히기로 했습니다. 김 부총리는 신북방·신남방 정책을 구체화하고 중동과 중남미 시장도 적극 개척하겠다고 강조했습니다. [김동연/경제부총리 : 보다 근본적으로 우리 경제가 대외 통상 마찰에도 크게 흔들림이 없도록 경제 협력의 지평을 넓히기 위한 최선을 다해 나갈 계획입니다.] 정부는 일본이 주도하는 &환태평양경제동반자협정&, CPTPP가입 여부도 상반기 안에 결정하기로 했습니다. 환태평양경제동반자협정은 일본, 호주, 뉴질랜드, 캐나다 등 11개국이 참가하는 자유무역협정으로, 해당국들은 지난 8일 협정에 정식 서명했습니다. 정부는 또 6주년을 맞는 한미FTA와 관련해선 국익 최우선의 원칙 아래 균형된 결과가 나오도록 개정 협상에 임하기로 했습니다. KBS 뉴스 김경진입니다.',
 'question': '미국의 관세 부과 조치에 대해서 정부는 어떻게 대응 할 계획인가?',
 'id': 'aihub-000028',
 'answers': {'answer_start': [276],
  'text':

In [109]:
len(train_10)

13219

In [110]:
len(dev_10)

1513

In [104]:
with open('/opt/ml/input/data/ai_hub/aihub_data.json','w') as outfile:
    json.dump(whole_list,outfile)

In [105]:
with open('/opt/ml/input/data/ai_hub/aihub_train_data.json','w') as outfile:
    json.dump(train_list,outfile)

In [106]:
with open('/opt/ml/input/data/ai_hub/aihub_validation_data.json','w') as outfile:
    json.dump(dev_list,outfile)

In [111]:
with open('/opt/ml/input/data/ai_hub/aihub_train_data10.json','w') as outfile:
    json.dump(train_10,outfile)

In [112]:
with open('/opt/ml/input/data/ai_hub/aihub_validation_data10.json','w') as outfile:
    json.dump(dev_10,outfile)

In [113]:
with open('/opt/ml/input/data/ai_hub/aihub_train_data.json','r') as outfile:
    tt= json.load(outfile)